# Data Wrangling Group Project

In [49]:
#install.packages("tidyverse")
#install.packages("skimr")
#install.packages("readr")
#install.packages("visdat")
library(skimr)
library(readr)
library(visdat)
library(tidyverse)

crash_df <- read_csv("https://raw.githubusercontent.com/zelta1990/STAT422-Group-Project/main/Crash_data_chch.csv") #Load Crash_data_chch.csv as a dataframe
crash_df %>% head() #check initial dataset
crash_df %>% glimpse() #check column types
crash_df <- crash_df %>% filter(crashYear > 2011) #only return records after 2012
#crash_df %>% skim() #quick data summary
#crash_df %>% vis_miss(large_data_size = 9e+07)#Visualise missing values

Rows: 51903 Columns: 72
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (20): crashDirectionDescription, crashFinancialYear, crashLocation1, cra...
dbl (50): X, Y, OBJECTID, advisorySpeed, areaUnitID, bicycle, bridge, bus, c...
lgl  (2): crashRoadSideRoad, intersection

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


X,Y,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,⋯,train,tree,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1569263,5177269,2,NA,595000,0,NA,0,2,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null
1554448,5181256,6,NA,587821,0,0,0,1,0,⋯,0,0,0,0,Open,0,0,1,Light rain,Null
1572260,5180996,8,NA,593300,0,NA,0,2,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null
1573557,5180808,16,NA,593600,0,NA,0,2,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null
1573407,5180049,22,NA,593600,0,NA,0,1,NA,⋯,NA,NA,0,0,Urban,1,NA,NA,Heavy rain,Null
1568675,5183354,27,NA,592100,0,NA,0,3,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null


Rows: 51,903
Columns: 72
$ X                         <dbl> 1569263, 1554448, 1572260, 1573557, 1573407,~
$ Y                         <dbl> 5177269, 5181256, 5180996, 5180808, 5180049,~
$ OBJECTID                  <dbl> 2, 6, 8, 16, 22, 27, 30, 36, 37, 38, 44, 46,~
$ advisorySpeed             <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, ~
$ areaUnitID                <dbl> 595000, 587821, 593300, 593600, 593600, 5921~
$ bicycle                   <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~
$ bridge                    <dbl> NA, 0, NA, NA, NA, NA, NA, 0, NA, NA, NA, NA~
$ bus                       <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~
$ carStationWagon           <dbl> 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 2, 1, 0, 1,~
$ cliffBank                 <dbl> NA, 0, NA, NA, NA, NA, NA, 0, NA, NA, NA, NA~
$ crashDirectionDescription <chr> "East", "West", "North", "Null", "Null", "No~
$ crashFinancialYear        <chr> "1999/2000", "1999/2000", "2000/2001", "2000~
$ crashLocation

In [41]:
#Calculate the count of accidents at one location for each year
count_df <- crash_df %>% 
                filter(crashSeverity == "Serious Crash" | crashSeverity == "Fatal Crash" ) %>%
                count(crashLocation1, crashLocation2, crashYear,sort=TRUE) 
#Rank each location by accident count for each year
all_year = list()
for(i in unique(crash_df$crashYear))
{
    year_df <- count_df %>% 
                    filter(crashYear == i ) %>% #only return locations with accident count > 1
                    arrange(desc(n)) 
    all_year[[i]] <- year_df
}
all_year_df <- bind_rows(all_year)
all_year_df
#Using 2021 as an example
#all_year_df %>% filter(crashYear == '2021') 



crashLocation1,crashLocation2,crashYear,n
<chr>,<chr>,<dbl>,<int>
SH 74,RADCLIFFE ROAD,2012,3
MERRIN ST,WITHELLS ROAD,2012,2
SH 75,KINLOCH ROAD S,2012,2
SH 76,COLOMBO ST,2012,2
BARBADOES ST,MOORHOUSE AVENUE,2012,1
BARRINGTON ST,BARRINGTON OFF WBD,2012,1
BARRINGTON ST,JERROLD ST N,2012,1
BARRINGTON ST,KINVER PLACE,2012,1
BARRINGTON ST,NEVILLE ST,2012,1


In [50]:
#Calculate the accumulative count of serious or fatal accidents at one location over ten years and return locations with count > 1
count_all_year_df <- crash_df %>% 
                filter(crashSeverity == "Serious Crash" | crashSeverity == "Fatal Crash" ) %>%
                count(crashLocation1, crashLocation2,sort=TRUE) 
count_all_year_df <- count_all_year_df %>%
                        filter(n > 1) 
count_all_year_df


crashLocation1,crashLocation2,n
<chr>,<chr>,<int>
GEBBIES PASS ROAD,SUMMIT ROAD,9
DYERS PASS ROAD,SUMMIT ROAD,7
SH 74,RADCLIFFE ROAD,7
DYERS PASS ROAD,CENTAURUS ROAD,6
HILLS ROAD,SHIRLEY ROAD,6
SH 74,MARSHLAND ROAD,6
GOVERNORS BAY ROAD,SANDY BEACH ROAD,5
CHRISTCHURCH AKAROA ROAD,CATONS CV,4
CRANFORD ST,FRASER ST,4


In [52]:
#Join count_all_year_df with original crash_df to return details for locations that had repeated accidents 
target_df <- inner_join(crash_df, count_all_year_df %>% select(-3), by = c('crashLocation1', 'crashLocation2')) 
                    
target_df 

X,Y,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,⋯,train,tree,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1569069,5185262,124814,NA,588102,0,NA,0,2,NA,⋯,NA,NA,0,0,Open,0,NA,NA,Fine,Null
1564295,5182423,128459,NA,589602,0,0,0,2,0,⋯,0,0,0,0,Urban,0,0,0,Fine,Null
1570878,5180886,128938,NA,591700,0,0,0,3,0,⋯,0,0,0,0,Urban,0,0,0,Fine,Null
1572933,5184220,129062,NA,592702,0,0,0,1,0,⋯,0,0,0,0,Urban,0,0,0,Fine,Null
1570500,5178327,129113,NA,594600,0,NA,0,2,NA,⋯,NA,NA,0,0,Urban,1,NA,NA,Fine,Null
1574466,5184474,129153,NA,590505,0,NA,0,1,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null
1569269,5184714,129262,NA,588102,0,0,0,5,0,⋯,0,0,0,0,Urban,0,0,0,Fine,Null
1570670,5190576,129318,NA,590400,0,NA,0,1,NA,⋯,NA,NA,1,0,Urban,0,NA,NA,Light rain,Null
1569448,5187955,129362,NA,590400,0,NA,0,2,NA,⋯,NA,NA,0,0,Urban,0,NA,NA,Fine,Null


In [21]:
#    Title: NZTM-WGS84(NZGD2000) converter
#    Author: mkennedy
#    Date: Feb 16, 2012 at 2:32
#    Availability: https://gis.stackexchange.com/questions/20389/converting-nzmg-or-nztm-to-latitude-longitude-for-use-with-r-map-library/20401#20401

#install.packages("proj4")
library(proj4)
get_lat_lon <- function(x,y)
{
    proj4string <- "+proj=tmerc +lat_0=0 +lon_0=173 +k=0.9996 +x_0=1600000 +y_0=10000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
    #proj4string <- "+proj=tmerc +lat_0=0.0 +lon_0=173.0 +k=0.9996 +x_0=1600000.0 +y_0=10000000.0 +datum=WGS84 +units=m"
    # Source data
    xy <- data.frame(x, y)

    # Transformed data
    pj <- project(xy, proj4string, inverse=TRUE)
    latlon <- data.frame(lat=pj$y, lon=pj$x)
    return(latlon)
}
lat_lon <- get_lat_lon(1562170,5178795)
lat_lon
lat_lon <- get_lat_lon(1560760,5178564)
lat_lon
lat_lon <- get_lat_lon(1561689,5178744)
lat_lon




lat,lon
<dbl>,<dbl>
-43.54283,172.5317


lat,lon
<dbl>,<dbl>
-43.54484,172.5143


lat,lon
<dbl>,<dbl>
-43.54327,172.5258


In [ ]:
#Plotting interactive map using leaflet
#install.packages("sp")
library(sp)
library(leaflet)
all_event_df$Lat <- as.numeric(all_event_df$Lat)
all_event_df$Lon <- as.numeric(all_event_df$Lon)
df <- all_event_df %>%
        select(Lat,Lon,Genre,Venue,Name,Address)
df <- na.omit(df)
event_palette <- colorFactor( palette="RdYlBu", domain=df$Genre, na.color="transparent")

# Prepare the text for the tooltip:
hover_text <- paste(
   "Name: ", df$Name, "<br/>",
   "Genre: ", df$Genre, "<br/>",
   "Venue: ", df$Venue, "<br/>") %>%
  lapply(htmltools::HTML)
leaflet(df) %>% 
  addTiles()  %>% 
  setView( lat=-43.5, lng=172.6 , zoom=10) %>%
  addCircleMarkers(~Lon, ~Lat, 
    fillColor = ~event_palette(Genre), fillOpacity = 0.7, color="#af4343", radius=8, stroke=FALSE,
    label = hover_text,
    labelOptions = labelOptions( style = list("font-weight" = "normal", padding = "3px 8px"), textsize = "13px", direction = "auto")
  ) %>%
  addLegend(pal=event_palette, values=~Genre, opacity=0.9, title = "Genre", position = "bottomright" )